In [1]:
class Opion():
    
    def __init__(self):
            
        self.dataroot= r'I:\irregular holes\paris_eval_gt' #image dataroot
        self.s1_dataroot= r'I:\irregular holes\paris_eval_gt' #sent1 dataroot
        self.maskroot= r'I:\irregular holes\testing_mask_dataset'#mask dataroot
        self.batchSize= 1   # Need to be set to 1
        self.fineSize=512 # image size
        self.input_nc=16  # input channel size for first stage
        self.input_nc_g=28 # input channel size for second stage
        self.output_nc=12# output channel size
        self.ngf=64 # inner channel
        self.ndf=64# inner channel
        self.which_model_netD='basic' # patch discriminator
        
        self.which_model_netF='feature'# feature patch discriminator
        self.which_model_netG='unet_csa'# seconde stage network
        self.which_model_netP='unet_256'# first stage network
        self.triple_weight=1
        self.name='CSA_inpainting'
        self.n_layers_D='3' # network depth
        self.gpu_ids=[0]
        self.model='csa_net'
        self.checkpoints_dir=r'.\checkpoints' #
        self.pretrained_dir = 'pretrained/vgg16_s2_weights.pth'
        self.norm='instance'
        self.fixed_mask=1
        self.use_dropout=False
        self.init_type='normal'
        self.mask_type='manual'
        self.lambda_A=100
        self.threshold=5/16.0
        self.stride=1
        self.shift_sz=1 # size of feature patch
        self.mask_thred=1
        self.bottleneck=512
        self.gp_lambda=10.0
        self.ncritic=5
        self.constrain='MSE'
        self.strength=1
        self.init_gain=0.02
        self.cosis=1
        self.gan_type='lsgan'
        self.gan_weight=0.2
        self.overlap=4
        self.skip=0
        self.display_freq=1000
        self.print_freq=50
        self.save_latest_freq=5000
        self.save_epoch_freq=2
        self.continue_train=False
        self.epoch_count=1
        self.phase='train'
        self.which_epoch=''
        self.niter=20
        self.niter_decay=100
        self.beta1=0.5
        self.lr=0.0002
        self.lr_policy='lambda'
        self.lr_decay_iters=50
        self.isTrain=True

In [2]:
import time
from util.data_load import Data_load
from models.models import create_model
import torch
import os
import numpy as np
import torchvision
from torch.utils import data
import torchvision.transforms as transforms

In [3]:
# seed = np.random.randint(0, 151981651)
# seed = 0
# torch.manual_seed(seed)
tensor = torch.rand((12, 512, 512), dtype=torch.float32)*65535
tensor2 = tensor.clone()

In [24]:
rand = np.random.choice([0, 1])
data_cast = transforms.Lambda(lambda x: (torch.clamp(x, 0, 65535)/65535*255).type(torch.uint8))
t = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(rand),
    transforms.ToTensor()
                        ])
normalization = [transforms.Normalize(mean=[0.5] * n, std=[0.5] * n) for n in [12, 4]]

tr_tensor = data_cast(tensor)
tr_tensor2 = data_cast(tensor2)
tr_tensor = torch.cat(tuple(t(tr_tensor[n, :, :]) for n in range(tr_tensor.size(0))), 0)
tr_tensor2 = torch.cat(tuple(t(tr_tensor2[n, :, :]) for n in range(tr_tensor2.size(0))), 0)

print((tr_tensor == tr_tensor2).all())
# tensor == tensor2
# print(tensor/tensor.max([1, 2], True)*255)
# data_cast(tr_tensor)
normalization[0](tr_tensor)[0, ...].max()

tensor(True)


tensor(0.9922)

In [ ]:
opt = Opion()
rand = np.random.choice([0, 1])

# TODO: Must find what minimum and maximum values Sentinel 1 images take
data_cast = [transforms.Lambda(lambda x: (torch.clamp(x, min_, max_)/max_*255).type(torch.uint8)) for min_, max_ in [(0, 65535), (-20, 30)]]
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(rand),
    transforms.ToTensor()
                        ])
normalization = [transforms.Normalize(mean=[0.5] * n, std=[0.5] * n) for n in [12, 4]]

dataset_train = Data_load(opt.dataroot, opt.maskroot, opt.s1_dataroot, transform, normalization, data_cast)
iterator_train = (data.DataLoader(dataset_train, batch_size=opt.batchSize,shuffle=True))
print(len(dataset_train))
model = create_model(opt)
total_steps = 0


In [ ]:
iter_start_time = time.time()
for epoch in range(opt.epoch_count, opt.niter + opt.niter_decay + 1):
    
    epoch_start_time = time.time()
    epoch_iter = 0

#     image, mask, gt = [x.cuda() for x in next(iterator_train)]
    for image, mask, sent1 in (iterator_train):
        image=image.cuda()
        mask=mask.cuda()
        sent1=sent1.cuda()
        mask=mask[0][0]
        mask=torch.unsqueeze(mask,0)
        mask=torch.unsqueeze(mask,1)
        mask=mask.byte()

        total_steps += opt.batchSize
        epoch_iter += opt.batchSize
        model.set_input(image,mask,sent1) # it not only sets the input data with mask, but also sets the latent mask.
        model.set_gt_latent()
        model.optimize_parameters()

        if total_steps %opt.display_freq== 0:
            real_A,real_B,fake_B=model.get_current_visuals()
            #real_A=input, real_B=ground truth fake_b=output
            pic = (torch.cat([real_A, real_B,fake_B], dim=0) + 1) / 2.0
            torchvision.utils.save_image(pic, '%s/Epoch_(%d)_(%dof%d).jpg' % (
            save_dir, epoch, total_steps + 1, len(dataset_train)), nrow=2) # Define save_dir
        if total_steps %1== 0:
            errors = model.get_current_errors()
            t = (time.time() - iter_start_time) / opt.batchSize
            print(errors)

    if epoch % opt.save_epoch_freq == 0:
        print('saving the model at the end of epoch %d, iters %d' %
                (epoch, total_steps))
        model.save(epoch)

    print('End of epoch %d / %d \t Time Taken: %d sec' %
            (epoch, opt.niter + opt.niter_decay, time.time() - epoch_start_time))

    model.update_learning_rate()
